# Covered Calls Leap Trader

## Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio
import yfinance as yf

from dateutil.relativedelta import relativedelta

from tqdm import tqdm
tqdm.pandas()

## Params

In [2]:
# Difference between long and short term gains (federal change shown below).
# However state by state laws are different... Feel free to look at 
# Input `TAX_ADVANTAGE` as a percent
TAX_ADVANTAGE = .17

# Input `TAX_RATE` as a percent
TAX_RATE = 0.37 + .1075

# Score risk tolerance by standard deviations (cumulative coverage percentage below)
RISK_TOLERANCE = 1

# How much each option takes to trade (this number is based of Schwab)
FEE = .65

# Get options data `AFTER_DATE`
AFTER_DATE = pd.to_datetime('2000-01-01')
BEFORE_DATE = pd.to_datetime('2025-08-08')

# Sets a maximum days to expiration
MAX_DAYS_TILL_EXPIRATION = 400

# Sets the ticker to trade
TICKERS = ['QQQ', 'SPY']

## Read In File

In [3]:
def pick_closest(group):
    out = []
    for num_weeks in [1, 2, 4, 8, 16, 32, 52]:
        target = group.name + relativedelta(weeks=num_weeks)
        # find the expiration_date closest to target
        closest_exp = group.loc[
            (group['expiration_date'] - target).abs().idxmin(),
            'expiration_date'
        ]
        # grab all rows with that expiration_date
        sub = group[group['expiration_date'] == closest_exp].copy()
        sub['expiration_group'] = num_weeks
        out.append(sub)
    # concatenate all horizons into one DataFrame
    return pd.concat(out, ignore_index=True)


In [4]:
# Initialize a dictionary to hold dataframes for each ticker
all_options_data = {}
portfolio_option_data = {}
portfolio_stock_metric = {}
portfolio_stock_data = {}

# Iterate through tickers and set dictionary to hold proper data
for TICKER in TICKERS:
    
    print(TICKER)
    
    # Read data
    all_options_data[TICKER] = pd.read_parquet(f'../read_data/data/clean/{TICKER}.parquet')
    portfolio_stock_data[TICKER] = all_options_data[TICKER].groupby('date')['stock_price'].first().sort_index().reset_index()

    # Stock data
    portfolio_stock_metric[TICKER] = portfolio_stock_data[TICKER].copy()
    portfolio_stock_metric[TICKER]['RSI_14']  = ta.rsi(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=14
    )

    # Simple & Exponential Moving Averages (20-day)
    portfolio_stock_metric[TICKER]['SMA_20'] = ta.sma(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=20
    )
    portfolio_stock_metric[TICKER]['EMA_20'] = ta.ema(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=20
    )

    # MACD (12,26,9): line, signal, histogram
    macd = ta.macd(
        portfolio_stock_metric[TICKER]['stock_price'],
        fast=12, slow=26, signal=9
    )
    portfolio_stock_metric[TICKER]['MACD'] = macd['MACD_12_26_9']
    portfolio_stock_metric[TICKER]['MACD_signal'] = macd['MACDs_12_26_9']
    portfolio_stock_metric[TICKER]['MACD_hist'] = macd['MACDh_12_26_9']

    portfolio_stock_metric[TICKER]['high_1w'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=5).max()
    portfolio_stock_metric[TICKER]['low_1w']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=5).min()

    portfolio_stock_metric[TICKER]['high_1m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=21).max()
    portfolio_stock_metric[TICKER]['low_1m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=21).min()

    portfolio_stock_metric[TICKER]['high_3m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=63).max()
    portfolio_stock_metric[TICKER]['low_3m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=63).min()

    portfolio_stock_metric[TICKER]['high_6m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=126).max()
    portfolio_stock_metric[TICKER]['low_6m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=126).min()

    portfolio_stock_metric[TICKER]['high_9m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=189).max()
    portfolio_stock_metric[TICKER]['low_9m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=189).min()

    portfolio_stock_metric[TICKER]['high_1y'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=252).max()
    portfolio_stock_metric[TICKER]['low_1y']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=252).min()
    portfolio_stock_metric[TICKER] = portfolio_stock_metric[TICKER].dropna()
    portfolio_stock_metric[TICKER] = portfolio_stock_metric[TICKER].drop(columns=['stock_price'])

    portfolio_option_data[TICKER] = all_options_data[TICKER].loc[all_options_data[TICKER]['date'].isin(portfolio_stock_metric[TICKER]['date'])]

    portfolio_option_data[TICKER] = portfolio_option_data[TICKER].loc[portfolio_option_data[TICKER]['days_till_expiration'] < MAX_DAYS_TILL_EXPIRATION].groupby('date').progress_apply(pick_closest)
    portfolio_option_data[TICKER] = portfolio_option_data[TICKER].reset_index(drop=True)



QQQ


100%|██████████| 2766/2766 [00:10<00:00, 266.72it/s]


SPY


100%|██████████| 3249/3249 [00:11<00:00, 271.85it/s]


In [ ]:
portfolio_calls = {}
portfolio_puts = {}

for TICKER in TICKERS:

    print(TICKER)

    portfolio_calls[TICKER] = pd.concat([
        portfolio_option_data[TICKER].loc[
            portfolio_option_data[TICKER]['stock_price'] > portfolio_option_data[TICKER]['strike']
        ].groupby(['date', 'expiration_group'], group_keys=False).progress_apply(lambda grp: grp.nsmallest(5, 'strike_distance')),
        portfolio_option_data[TICKER].loc[
            portfolio_option_data[TICKER]['stock_price'] < portfolio_option_data[TICKER]['strike']
        ].groupby(['date', 'expiration_group'], group_keys=False).progress_apply(lambda grp: grp.nsmallest(5, 'strike_distance'))
    ])[
        ['date', 'strike', 'days_till_expiration', 'expiration_date', 'stock_price', 'strike_distance', 'call_price', 'expiration_group'] +
        [col for col in portfolio_option_data[TICKER].columns if col.startswith('c_')]
    ].rename(
        columns=lambda col: (
            col[2:]
            if col.startswith('c_') 
            else ('price' if col.endswith('_price') else col)
        )
    ).drop(columns=['volume', 'last', 'size', 'bid', 'ask'], errors='ignore').merge(
          portfolio_stock_metric[TICKER],
          on='date',
          how='left'
    )

    portfolio_puts[TICKER] = portfolio_option_data[TICKER].loc[
        portfolio_option_data[TICKER]['stock_price'] < portfolio_option_data[TICKER]['strike']
    ].groupby(['date', 'expiration_group'], group_keys=False).progress_apply(lambda grp: grp.nsmallest(10, 'strike_distance'))[
        ['date', 'strike', 'days_till_expiration', 'expiration_date', 'stock_price', 'strike_distance', 'put_price', 'expiration_group'] +
        [col for col in portfolio_option_data[TICKER].columns if col.startswith('p_')]
    ].rename(
        columns=lambda col: (
            col[2:]
            if col.startswith('p_') 
            else ('price' if col.endswith('_price') else col)
        )
    ).drop(columns=['volume', 'last', 'size', 'bid', 'ask'], errors='ignore').merge(
          portfolio_stock_metric[TICKER],
          on='date',
          how='left'
    )

QQQ


100%|██████████| 19328/19328 [00:09<00:00, 2080.78it/s]


SPY


100%|██████████| 22742/22742 [00:11<00:00, 2027.35it/s]


In [33]:
# Additional imports for RL
import gym
from gym import spaces
import stable_baselines3 as sb3
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from collections import deque
from typing import List, Dict, Tuple, Optional, Any
import warnings
warnings.filterwarnings('ignore')

def preprocess_options_data(
    portfolio_calls: Dict[str, pd.DataFrame], 
    portfolio_puts: Dict[str, pd.DataFrame], 
    ticker: str
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Concatenate calls and puts into a single dataframe with consistent N options per date.
    
    Args:
        portfolio_calls: Dictionary of call options dataframes by ticker
        portfolio_puts: Dictionary of put options dataframes by ticker  
        ticker: Ticker symbol to process
        
    Returns:
        Tuple of (all_options_df, dates_list)
    """
    # Add option type identifier
    calls_df = portfolio_calls[ticker].copy()
    calls_df['option_type'] = 'call'
    calls_df['option_side'] = 'call'
    
    puts_df = portfolio_puts[ticker].copy()
    puts_df['option_type'] = 'put'
    puts_df['option_side'] = 'put'
    
    # Ensure both have the same columns for concatenation
    common_cols = set(calls_df.columns) & set(puts_df.columns)
    calls_df = calls_df[list(common_cols)]
    puts_df = puts_df[list(common_cols)]
    
    # Concatenate calls and puts
    all_options = pd.concat([calls_df, puts_df], ignore_index=True)
    
    # Keep only essential columns for trading
    essential_cols = [
        'date', 'strike', 'days_till_expiration', 'expiration_date', 
        'stock_price', 'strike_distance', 'price', 'expiration_group',
        'option_type', 'option_side'
    ]
    
    # Add Greek columns if they exist
    greek_cols = ['implied_volatility', 'delta', 'theta', 'gamma', 'vega', 'rho']
    for col in greek_cols:
        if col in all_options.columns:
            essential_cols.append(col)
    
    # Add bid/ask if available
    if 'bid' in all_options.columns and 'ask' in all_options.columns:
        essential_cols.extend(['bid', 'ask'])
    elif 'bid' not in all_options.columns and 'ask' not in all_options.columns:
        # Create mock bid/ask from price (assuming 1% spread)
        all_options['bid'] = all_options['price'] * 0.995
        all_options['ask'] = all_options['price'] * 1.005
        essential_cols.extend(['bid', 'ask'])
    
    all_options = all_options[essential_cols]
    
    # Sort by date, expiration_group, option_type, strike_distance for consistent ordering
    all_options = all_options.sort_values([
        'date', 'expiration_group', 'option_type', 'strike_distance'
    ]).reset_index(drop=True)
    
    # Get unique dates
    dates = sorted(all_options['date'].unique())
    
    # Verify we have consistent number of options per date
    options_per_date = all_options.groupby('date').size()
    if not options_per_date.nunique() == 1:
        print(f"Warning: Inconsistent number of options per date. Range: {options_per_date.min()}-{options_per_date.max()}")
    
    return all_options, dates

class OptionsTradingEnv(gym.Env):
    """
    Gymnasium environment for options trading with reinforcement learning.
    
    Action space: MultiDiscrete([3]*M) where M is number of options
    - 0: Short position (-1)
    - 1: Hold/No position (0)  
    - 2: Long position (+1)
    
    Observation space: Box containing:
    - Stock metrics (RSI, MACD, moving averages, etc.)
    - Option features (implied_volatility, delta, theta, ask price, etc.) for each option
    - Current positions for each option
    """
    
    def __init__(
        self,
        dates: List[str],
        stock_metrics_df: pd.DataFrame,
        all_options_df: pd.DataFrame,
        fee: float = 0.65,
        risk_lambda: float = 0.1,
        lookback_window: int = 30
    ):
        """
        Initialize the options trading environment.
        
        Args:
            dates: List of trading dates
            stock_metrics_df: DataFrame with stock technical indicators indexed by date
            all_options_df: DataFrame with option data, must have consistent options per date
            fee: Transaction fee per option contract
            risk_lambda: Risk aversion parameter for reward calculation
            lookback_window: Number of periods to look back for risk calculation
        """
        super().__init__()
        
        self.dates = dates
        self.stock_metrics_df = stock_metrics_df.set_index('date') if 'date' in stock_metrics_df.columns else stock_metrics_df
        self.all_options_df = all_options_df
        self.fee = fee
        self.risk_lambda = risk_lambda
        self.lookback_window = lookback_window
        
        # Determine number of options per date
        options_per_date = self.all_options_df.groupby('date').size()
        self.num_options = options_per_date.iloc[0]
        
        # Define action space: 3 actions (short, hold, long) for each option
        self.action_space = spaces.MultiDiscrete([3] * self.num_options)
        
        # Define observation space dimensions
        stock_features = len(self.stock_metrics_df.columns)
        
        # Option features: implied_volatility, delta, theta, ask price, etc.
        option_sample = self.all_options_df[self.all_options_df['date'] == self.dates[0]]
        self.option_features = ['implied_volatility', 'delta', 'theta', 'ask', 'strike_distance', 'days_till_expiration']
        self.option_features = [f for f in self.option_features if f in option_sample.columns]
        
        option_features_dim = len(self.option_features) * self.num_options
        positions_dim = self.num_options
        
        total_dim = stock_features + option_features_dim + positions_dim
        
        # Box observation space (continuous values)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(total_dim,), dtype=np.float32
        )
        
        # Initialize state
        self.current_step = 0
        self.positions = np.zeros(self.num_options)  # -1: short, 0: hold, 1: long
        self.pnl_history = deque(maxlen=self.lookback_window)
        self.portfolio_value = 0.0
        
        print(f"Environment initialized with {self.num_options} options per date")
        print(f"Observation space: {self.observation_space.shape}")
        print(f"Action space: {self.action_space}")
        print(f"Option features: {self.option_features}")
        
    def reset(self, seed: Optional[int] = None, options: Optional[Dict[str, Any]] = None) -> Tuple[np.ndarray, Dict[str, Any]]:
        """Reset the environment to initial state."""
        if seed is not None:
            np.random.seed(seed)
            
        self.current_step = 0
        self.positions = np.zeros(self.num_options)
        self.pnl_history.clear()
        self.portfolio_value = 0.0
        
        obs = self._get_obs()
        info = {'date': self.dates[self.current_step]}
        
        return obs, info
    
    def step(self, action: np.ndarray) -> Tuple[np.ndarray, float, bool, bool, Dict[str, Any]]:
        """
        Execute one step in the environment.
        
        Args:
            action: Array of actions for each option (0=short, 1=hold, 2=long)
            
        Returns:
            Tuple of (observation, reward, terminated, truncated, info)
        """
        # Convert action to position changes
        new_positions = np.array(action) - 1  # Convert 0,1,2 to -1,0,1
        
        # Calculate position changes and transaction costs
        position_changes = new_positions - self.positions
        transaction_cost = np.sum(np.abs(position_changes)) * self.fee
        
        # Update positions
        self.positions = new_positions
        
        # Calculate mark-to-market PnL
        option_prices = self._get_option_prices()
        
        # PnL = positions * option_prices (simplified, assumes we can trade at mid price)
        current_pnl = np.sum(self.positions * option_prices) - transaction_cost
        
        # Add to PnL history
        self.pnl_history.append(current_pnl)
        
        # Calculate reward: PnL - risk penalty
        if len(self.pnl_history) >= 2:
            pnl_std = np.std(list(self.pnl_history))
            risk_penalty = self.risk_lambda * pnl_std
        else:
            risk_penalty = 0.0
        
        reward = current_pnl - risk_penalty
        
        # Update portfolio value
        self.portfolio_value += current_pnl
        
        # Advance to next step
        self.current_step += 1
        
        # Check if episode is done
        terminated = self.current_step >= len(self.dates) - 1
        truncated = False
        
        # Get new observation
        obs = self._get_obs() if not terminated else np.zeros(self.observation_space.shape)
        
        info = {
            'date': self.dates[self.current_step] if not terminated else self.dates[-1],
            'pnl': current_pnl,
            'portfolio_value': self.portfolio_value,
            'transaction_cost': transaction_cost,
            'risk_penalty': risk_penalty,
            'positions': self.positions.copy()
        }
        
        return obs, reward, terminated, truncated, info
    
    def _get_obs(self) -> np.ndarray:
        """Get current observation."""
        current_date = self.dates[self.current_step]
        
        # Get stock metrics
        stock_metrics = self.stock_metrics_df.loc[current_date].values
        
        # Get option features for current date
        current_options = self.all_options_df[self.all_options_df['date'] == current_date]
        current_options = current_options.sort_values(['expiration_group', 'option_type', 'strike_distance'])
        
        # Extract option features
        option_features = []
        for feature in self.option_features:
            if feature in current_options.columns:
                option_features.extend(current_options[feature].values)
            else:
                option_features.extend([0.0] * self.num_options)
        
        # Combine all features
        obs = np.concatenate([
            stock_metrics,
            option_features,
            self.positions
        ]).astype(np.float32)
        
        return obs
    
    def _get_option_prices(self) -> np.ndarray:
        """Get mid prices for current date's options."""
        current_date = self.dates[self.current_step]
        current_options = self.all_options_df[self.all_options_df['date'] == current_date]
        current_options = current_options.sort_values(['expiration_group', 'option_type', 'strike_distance'])
        
        # Use mid price if bid/ask available, otherwise use price
        if 'bid' in current_options.columns and 'ask' in current_options.columns:
            mid_prices = (current_options['bid'] + current_options['ask']) / 2
        else:
            mid_prices = current_options['price']
        
        return mid_prices.values

def create_training_env(
    ticker: str,
    portfolio_calls: Dict[str, pd.DataFrame],
    portfolio_puts: Dict[str, pd.DataFrame], 
    portfolio_stock_metric: Dict[str, pd.DataFrame],
    train_start_date: str = None,
    train_end_date: str = None,
    fee: float = 0.65,
    risk_lambda: float = 0.1
) -> OptionsTradingEnv:
    """
    Create a training environment for a specific ticker.
    
    Args:
        ticker: Stock ticker symbol
        portfolio_calls: Dictionary of call options dataframes
        portfolio_puts: Dictionary of put options dataframes
        portfolio_stock_metric: Dictionary of stock metrics dataframes
        train_start_date: Start date for training (optional)
        train_end_date: End date for training (optional)
        fee: Transaction fee per contract
        risk_lambda: Risk aversion parameter
        
    Returns:
        OptionsTradingEnv instance
    """
    # Preprocess options data
    all_options, all_dates = preprocess_options_data(portfolio_calls, portfolio_puts, ticker)
    
    # Filter dates for training if specified
    if train_start_date or train_end_date:
        date_mask = pd.Series(True, index=range(len(all_dates)))
        if train_start_date:
            date_mask &= pd.Series(all_dates) >= train_start_date
        if train_end_date:
            date_mask &= pd.Series(all_dates) <= train_end_date
        
        training_dates = [all_dates[i] for i in range(len(all_dates)) if date_mask.iloc[i]]
        training_options = all_options[all_options['date'].isin(training_dates)]
    else:
        training_dates = all_dates
        training_options = all_options
    
    # Create environment
    env = OptionsTradingEnv(
        dates=training_dates,
        stock_metrics_df=portfolio_stock_metric[ticker],
        all_options_df=training_options,
        fee=fee,
        risk_lambda=risk_lambda
    )
    
    return env


In [34]:
# Example: Create and use the environment with QQQ data

# Choose ticker for training
TICKER = 'QQQ'

# Create the environment
print("Creating environment...")
env = create_training_env(
    ticker=TICKER,
    portfolio_calls=portfolio_calls,
    portfolio_puts=portfolio_puts,
    portfolio_stock_metric=portfolio_stock_metric,
    train_start_date='2010-01-01',  # Optional: filter training dates
    train_end_date='2023-12-31',    # Optional: filter training dates
    fee=FEE,                        # Use the fee from parameters
    risk_lambda=0.1                 # Risk aversion parameter
)

# Validate environment
print("\nValidating environment...")
try:
    check_env(env)
    print("✓ Environment validation passed!")
except Exception as e:
    print(f"✗ Environment validation failed: {e}")

# Test a few steps manually
print("\nTesting environment...")
obs, info = env.reset()
print(f"Initial observation shape: {obs.shape}")
print(f"Initial date: {info['date']}")

# Take a random action
action = env.action_space.sample()
print(f"Random action: {action}")

obs, reward, terminated, truncated, info = env.step(action)
print(f"Step result - Reward: {reward:.4f}, Date: {info['date']}")
print(f"Portfolio value: {info['portfolio_value']:.4f}")
print(f"Transaction cost: {info['transaction_cost']:.4f}")

# Reset for fresh start
obs, info = env.reset()
print(f"\nEnvironment reset. Ready for training!")

# Print some useful info about the environment
print(f"\nEnvironment details:")
print(f"- Number of options per date: {env.num_options}")
print(f"- Total training dates: {len(env.dates)}")
print(f"- Option features used: {env.option_features}")
print(f"- Stock metrics: {list(env.stock_metrics_df.columns)}")
print(f"- Action space: {env.action_space}")
print(f"- Observation space: {env.observation_space}")

# Feature normalization suggestion
print(f"\n💡 Feature Normalization Suggestions:")
print(f"Consider normalizing features like:")
print(f"- Stock prices and technical indicators (RSI, MACD, etc.)")
print(f"- Option Greeks (delta, theta, vega)")
print(f"- Implied volatility")
print(f"- Strike distances")
print(f"This can significantly improve training performance!")

# Train/test split suggestion
print(f"\n💡 Train/Test Split Suggestions:")
print(f"- Use first 80% of dates for training")
print(f"- Use last 20% for testing")
print(f"- Consider walk-forward validation for time series data")
print(f"- Example: train_end_date='2020-12-31', test_start_date='2021-01-01'")

# Risk lambda tuning suggestion
print(f"\n💡 Risk Lambda (λ) Tuning:")
print(f"- λ = 0.0: No risk penalty (pure profit maximization)")
print(f"- λ = 0.1: Moderate risk aversion")
print(f"- λ = 0.5: High risk aversion")
print(f"- λ = 1.0: Very high risk aversion")
print(f"- Tune this parameter based on your risk tolerance")


Creating environment...


ValueError: Cannot set a DataFrame with multiple columns to the single column bid

In [ ]:
# Stub training function - uncomment and modify as needed
def train_options_model():
    """
    Example training function using DQN.
    Uncomment and modify the code below to actually train a model.
    """
    print("Training function placeholder")
    print("Uncomment the training code below to start training!")
    
    # # Create DQN model
    # model = DQN(
    #     policy='MlpPolicy',
    #     env=env,
    #     learning_rate=1e-4,
    #     buffer_size=50000,
    #     learning_starts=5000,
    #     batch_size=32,
    #     gamma=0.99,
    #     verbose=1
    # )
    
    # # Train the model
    # model.learn(total_timesteps=50000)
    
    # # Save the model
    # model.save("options_trading_model")
    # print("Model saved!")
    
    # return model

# Call the training function
train_options_model()

print("\n" + "="*50)
print("TRAINING INSTRUCTIONS")
print("="*50)

print("\n💡 To actually train a model:")
print("1. Uncomment the training code in the train_options_model() function above")
print("2. Adjust hyperparameters as needed")
print("3. Run the cell to start training")

print("\n📊 Monitor training with TensorBoard:")
print("   tensorboard --logdir=./logs/")

print("\n🎯 Alternative RL Algorithms to try:")
print("- PPO: from stable_baselines3 import PPO")
print("- A2C: from stable_baselines3 import A2C")
print("- SAC: from stable_baselines3 import SAC")

print("\n⚙️ Key Hyperparameters:")
print("- learning_rate: 1e-3, 1e-4, 1e-5")
print("- buffer_size: 10000, 50000, 100000")
print("- batch_size: 32, 64, 128")
print("- gamma: 0.95, 0.99, 0.999")

print("\n🔍 Evaluation:")
print("- Create test environment with unseen dates")
print("- Track: total return, Sharpe ratio, max drawdown")
print("- Compare against buy-and-hold baseline")

print("\n📈 Advanced Features:")
print("- Feature normalization for better training")
print("- Multi-ticker training")
print("- Position limits and risk constraints")
print("- Realistic transaction costs and slippage")
